In [1]:
import numpy as np
np.random.seed(37) 

In [2]:
# 准备数据集，建一个class来加载数据集，对数据进行预处理
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim import models, corpora

class DataSet:
    
    def __init__(self,txt_file_path):
        self.__txt_file=txt_file_path
    
    def __load_txt(self): # 从txt文档中加载文本内容，逐行读入
        with open(self.__txt_file,'r') as file:
            content=file.readlines() # 一次性将所有的行都读入
        return [line[:-1] for line in content] # 去掉每一行末尾的\n
    
    def __tokenize(self,lines_list): # 预处理之一：对每一行文本进行分词
        tokenizer=RegexpTokenizer('\w+') 
        # 此处用正则表达式分词器而不用word_tokenize的原因是：排除带有标点的单词
        return [tokenizer.tokenize(line.lower()) for line in lines_list]
    
    def __remove_stops(self,lines_list): # 预处理之二：对每一行取出停用词
        # 我们要删除一些停用词，避免这些词的噪声干扰，故而需要一个停用词表
        stop_words_list=stopwords.words('english')  # 获取英文停用词表
        return [[token for token in line if token not in stop_words_list]
                for line in lines_list] 
        # 这儿有点难以理解，lines_list含有的元素也是list，这一个list就是一行文本，
        # 而一行文本内部有N个分词组成，故而lines_list可以看出二维数组，需要用两层generator
    
    def __word_stemm(self,lines_list): # 预处理之三：对每个分词进行词干提取
        stemmer=SnowballStemmer('english')
        return [[stemmer.stem(word) for word in line] for line in lines_list]
    
    def prepare(self):
        '''供外部调用的函数，用于准备数据集'''
        # 先从txt文件中加载文本内容，再进行分词，再去除停用词，再进行词干提取
        stemmed_words=self.__word_stemm(self.__remove_stops(self.__tokenize(self.__load_txt())))
        # 后面的建模需要用到基于dict的词矩阵，故而先用corpora构建dict在建立词矩阵
        dict_words=corpora.Dictionary(stemmed_words)
        matrix_words=[dict_words.doc2bow(text) for text in stemmed_words]
        return dict_words, matrix_words 
    
    # 以下函数主要用于测试上面的几个函数是否运行正常
    def get_content(self):
        return self.__load_txt()
    
    def get_tokenize(self):
        return self.__tokenize(self.__load_txt())
    
    def get_remove_stops(self):
        return self.__remove_stops(self.__tokenize(self.__load_txt()))
    
    def get_word_stemm(self):
        return self.__word_stemm(self.__remove_stops(self.__tokenize(self.__load_txt())))

e:\programsoft\python\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# 检验上述DataSet类是否运行正常
dataset=DataSet("E:\PyProjects\DataSet\FireAI\data_topic_modeling.txt")

# 以下测试load_txt()函数是否正常
content=dataset.get_content()
print(len(content))
print(content[:3])

# 以下测试__tokenize()函数是否正常
tokenized=dataset.get_tokenize()
print(tokenized)

# 一下测试__remove_stops()函数是否正常
removed=dataset.get_remove_stops()
print(removed)

# 以下测试__word_stemm()函数是否正常
stemmed=dataset.get_word_stemm()
print(stemmed)

# 以下测试prepare函数是否正常
_,prepared=dataset.prepare()
print(prepared)

8
['He spent a lot of time studying cryptography. ', 'You need to have a very good understanding of modern encryption systems in order to work there.', "If their team doesn't win this match, they will be out of the competition."]
[['he', 'spent', 'a', 'lot', 'of', 'time', 'studying', 'cryptography'], ['you', 'need', 'to', 'have', 'a', 'very', 'good', 'understanding', 'of', 'modern', 'encryption', 'systems', 'in', 'order', 'to', 'work', 'there'], ['if', 'their', 'team', 'doesn', 't', 'win', 'this', 'match', 'they', 'will', 'be', 'out', 'of', 'the', 'competition'], ['those', 'codes', 'are', 'generated', 'by', 'a', 'specialized', 'machine'], ['the', 'club', 'needs', 'to', 'develop', 'a', 'policy', 'of', 'training', 'and', 'promoting', 'younger', 'talent'], ['his', 'movement', 'off', 'the', 'ball', 'is', 'really', 'great'], ['in', 'order', 'to', 'evade', 'the', 'defenders', 'he', 'needs', 'to', 'move', 'swiftly'], ['we', 'need', 'to', 'make', 'sure', 'only', 'the', 'authorized', 'parties',

In [4]:
# 以上测试表名DataSet类运行正常，
# 所准备的词矩阵也是符合预期，可以用于后续建模

In [5]:
# 获取数据集
dataset=DataSet("E:\PyProjects\DataSet\FireAI\data_topic_modeling.txt")
dict_words, matrix_words =dataset.prepare()

# 使用LDAModel建模
lda_model=models.ldamodel.LdaModel(matrix_words,num_topics=2,
                           id2word=dict_words,passes=25) 
# 此处假设原始文档有两个主题


In [6]:
# 查看模型中最重要的N个单词
print('Most important words to topics: ')
for item in lda_model.print_topics(num_topics=2,num_words=5):
    # 此处只打印最重要的5个单词
    print('Topic: {}, words: {}'.format(item[0],item[1]))

Most important words to topics: 
Topic: 0, words: 0.075*"need" + 0.053*"order" + 0.032*"system" + 0.032*"encrypt" + 0.032*"work"
Topic: 1, words: 0.037*"younger" + 0.037*"develop" + 0.037*"promot" + 0.037*"talent" + 0.037*"train"
